# Guild AI Interactive Python Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://guild.ai/docs/start/).

Guild AI is a tool for running, tracking, and comparing machine learning experiments. For more on Guild AI, visit https://guild.ai.

In this example, we show how Guild is used to run an unmodified Python function as a training operation. It uses a Pythonic interface defined in the module `guild.ipy`.

This notebook covers the following topics:

- Installation
- Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Hyperparameter search and optimization
    - Grid search
    - Random search
    - Bayesian optimization

## Installation

Before continuing, install Guild AI:

    $ pip install guildai
    
Install either `tensorflow` or `tensorflow-gpu` depending on your system capability:

    $ pip install tensorflow # (or tensorflow-gpu if your system has a GPU)
    
TensorFlow is required for TensorBoard support. Your training code does not need to use TensorFlow. This requirement will be removed in versions of Guild starting with `0.7`.
    
**<span style="color:#600">IMPORTANT</span>** If you are installing Guild AI 0.6.3, you must manually install a missing requirement (this bug is fixed in 0.6.4, which will be released in late May):

    $ pip install click
    
## Contents
    
This Notebook has two parts.

It features the module `guild.ipy`, which is a Python interface that can be used interactively in Notebooks or by your own Python code. 

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [35]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [36]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [37]:
import guild.ipy as guild

**<span style="color:#600">IMPORTANT</span>** If you get an error **No module named 'click'** install `click` manually by running `pip install click` in your Notebook environment.

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [38]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir("guild-env")
    
guild.set_guild_home("guild-env")

Clear the director of any runs.

In [39]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

Deleted 22 run(s)


## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [40]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

[Function credit: *skopt API documentation*](https://scikit-optimize.github.io/)

Run the mock training function a couple times to see how it works.

**NOTE** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [41]:
print("Trial 1:")
train(x=-2.0)

Trial 1:
loss: -0.068346


In [42]:
print("Trial 2:")
train(x=0.0)

Trial 2:
loss: 0.049435


In [43]:
print("Trial 3:")
train(x=2.0)

Trial 3:
loss: 0.059729


## Generate a run

Run the mock train function using Guild:

In [44]:
run, return_val = guild.run(train)

loss: 0.332362


The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [45]:
print(return_val)

None


The `run` variable is a Python object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [46]:
run.id

'd10558a07d9f11e9b16de4a471939b0d'

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [47]:
run.dir

'guild-env/runs/d10558a07d9f11e9b16de4a471939b0d'

Our mock training script doesn't generate any files, so the run directory is empty, with the exception of a `.guild` subdirectory.

In [48]:
os.listdir(run.dir)

['.guild']

The `.guild` subdirectory contains run output and metadata. Outputs include the output printed to the script during the operation (`output`) and any scalars logged (`events.*`). Guild logs scalar output using TensorFlow event files, which can be read using TensorBoard.

In [49]:
os.listdir(os.path.join(run.dir, ".guild"))

['output', 'opref', 'events.out.tfevents.1558646097.omaha', 'attrs']

Run metadata includes *attributes*. You can list the attribute names using `attr_names()`:

In [50]:
run.attr_names()

['exit_status', 'flags', 'id', 'initialized', 'started', 'stopped']

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [51]:
run.get("flags")

{'noise': 0.1, 'x': 0.1}

Flags are values provided to the function. In this case, both values are defined as function keyword default values. Later you run `train` using explicit flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [52]:
run.status

'completed'

## List runs

List runs using `guild.runs()`:

In [53]:
runs = guild.runs()
runs

,run,operation,started,status,label
0,d10558a0,train(),2019-05-23 16:14:57,completed,


Each time you run an operation, the run appears in the data frame generated by `runs()`.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [54]:
runs.info()

id: d10558a07d9f11e9b16de4a471939b0d
operation: train()
status: completed
started: 2019-05-23 16:14:57
stopped: 2019-05-23 16:14:57
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/d10558a07d9f11e9b16de4a471939b0d
flags:
  noise: 0.1
  x: 0.1


This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

First, you can specify the `scalars` flag to `info()`:

In [55]:
runs.info(scalars=True)

id: d10558a07d9f11e9b16de4a471939b0d
operation: train()
status: completed
started: 2019-05-23 16:14:57
stopped: 2019-05-23 16:14:57
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/d10558a07d9f11e9b16de4a471939b0d
flags:
  noise: 0.1
  x: 0.1
scalars:
  loss: 0.332362 (step 0)


Second, you can get a data frame containing scalars using the `scalars()` method.

In [56]:
scalars = runs.scalars()
scalars

,avg_val,count,first_step,first_val,last_step,last_val,max_step,max_val,min_step,min_val,prefix,run,tag,total
0,0.332362,1,0,0.332362,0,0.332362,0,0.332362,0,0.332362,,d10558a07d9f11e9b16de4a471939b0d,loss,0.332362


**<span style="color:#600">IMPORTANT</span>** If this listing is empty, ensure that you have TensorFlow installed (see above for instructions).

Guild stores aggregates of each tag, including *first*, *last*, *max*, *min*, and *average*.

You can use the various facilities in data frame to get scalar values.

In [57]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

0    0.332362
Name: last_val, dtype: float64

## Generate a second run

Run train again with different flags along with a run *label*:

In [58]:
_ = guild.run(train, x=0.2, _label="run 2")

loss: 0.908333


List runs:

In [59]:
guild.runs()

,run,operation,started,status,label
0,d93d37e0,train(),2019-05-23 16:15:10,completed,run 2
1,d10558a0,train(),2019-05-23 16:14:57,completed,


Note that the latest run contains the label "run 2" as specified in the `run()` call.

## Compare runs

Use `compare()` to generate a data frame containing both flags and scalars.

In [60]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,d93d37e0,train(),2019-05-23 16:15:10,0:00:00,completed,run 2,0.1,0.2,0,0.908333
1,d10558a0,train(),2019-05-23 16:14:57,0:00:00,completed,,0.1,0.1,0,0.332362


## Grid search

Grid search &mdash; also referred to as a parameter sweep &mdash; is a form of hyperparameter tuning that uses exhaustive search over a manually defined set of hyperparameter values.

To perform a grid search in Guild, provide a list of values to use for any given flag. If you specify lists for multiple flags, Guild runs trials for each possible flag value combination.

Run `train()` using a range of values:

In [61]:
_ = guild.run(train, x=[-0.5,-0.4,-0.3,-0.2,-0.1])

Running train (noise=0.1, x=-0.5):
loss: -0.620585
Running train (noise=0.1, x=-0.4):
loss: -0.840888
Running train (noise=0.1, x=-0.3):
loss: -1.015978
Running train (noise=0.1, x=-0.2):
loss: -0.773661
Running train (noise=0.1, x=-0.1):
loss: -0.526886


This command generates five trials, one for each specified value of `x`.

Use `runs().compare()[:5]` to compare the last five results:

In [62]:
guild.runs().compare()[:5]

,run,operation,started,time,status,label,noise,x,step,loss
0,da896d5c,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.1,0,-0.526886
1,da896d5b,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.2,0,-0.773661
2,da896d5a,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.3,0,-1.015978
3,da896d59,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.4,0,-0.840888
4,da896d58,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.5,0,-0.620585


View the top-three results by loss:

In [63]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
2,da896d5a,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.3,0,-1.015978
3,da896d59,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.4,0,-0.840888
1,da896d5b,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.2,0,-0.773661


Based on our mock training function, the "best" result (i.e. the run with the lowest *loss*) should be the run where `x` is close to `-0.3`. Because there's a random component (i.e. the `noise` parameter) your results may show best results with different values for `x`.

Below is an image that plots *loss* for values of *x*, showing the lowest loss where x is approximately `-0.3`.

<img src="bayesian-optimization.png" style="margin-left:0">

[Image credit: *Bayesian optimization with skopt*](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

## Random search

Random search is a method used in machine learning to explore hyperparameter spaces at random.

To run a series of runs using random values over a specified range, use the `slice` function to specify a range for a flag. To specify the number of trials, specify `_max_trials`. The default number of trials is `20`.

Run train five times with random values of `x` over the range `-2.0` to `2.0`.

In [64]:
_ = guild.run(train, x=slice(-2.0,2.0), _max_trials=5)

Running train (noise=0.1, x=1.2491154696153983):
loss: 0.025076
Running train (noise=0.1, x=1.26538516214455):
loss: 0.111353
Running train (noise=0.1, x=0.4020856284714598):
loss: 0.784366
Running train (noise=0.1, x=-1.828461247675734):
loss: 0.156651
Running train (noise=0.1, x=0.5696456975259441):
loss: 0.184666


Compare the results again.

In [65]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,dc3e49a6,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,0.569646,0,0.184666
1,dc3e49a5,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,-1.828461,0,0.156651
2,dc3e49a4,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,0.402086,0,0.784366
3,dc3e49a3,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,1.265385,0,0.111353
4,dc3e49a2,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,1.249115,0,0.025076
5,da896d5c,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.100000,0,-0.526886
6,da896d5b,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.200000,0,-0.773661
7,da896d5a,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.300000,0,-1.015978
8,da896d59,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.400000,0,-0.840888
9,da896d58,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.500000,0,-0.620585


List the three runs with the lowest loss:

In [66]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
7,da896d5a,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.3,0,-1.015978
8,da896d59,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.4,0,-0.840888
6,da896d5b,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.2,0,-0.773661


## Bayesian optimization

Bayesian optimizers use light weight models as surrogates for the target model — surrogates that can be evaluated quickly to recommend likely optimal hyperparameter values — and update those models using results from real trials.

Run `train` with Guild’s built-in Bayesian optimizer, which uses Gaussian processes. As with the earlier random search, we use Python's slice function to specify the range for `x` over which to search.

**NOTE:** The argument `bayesian` in the command is an alias for `gp`, which is a Bayesian optimizer that uses Gaussian processes. Guild supports three Bayesian optimizers: gp, forest and gbrt.

In [67]:
_ = guild.run(train, x=slice(-2.0,2.0), _optimizer="bayesian", _max_trials=10)

Running train (noise=0.1, x=1.556136206003075):
loss: 0.033814
Running train (noise=0.1, x=-2.0):
loss: 0.101926
Running train (noise=0.1, x=2.0):
loss: -0.036132
Running train (noise=0.1, x=-0.18699811915306785):
loss: -0.754781
Running train (noise=0.1, x=-0.2783458691539511):
loss: -0.795046
Running train (noise=0.1, x=-0.7996381776198571):
loss: 0.414261
Running train (noise=0.1, x=0.08720679270250331):
loss: 0.462514
Running train (noise=0.1, x=-1.99977889449307):
loss: 0.059006
Running train (noise=0.1, x=1.9995711109759342):
loss: -0.064552
Running train (noise=0.1, x=1.9999965526046442):
loss: 0.045584


View the top 10 results:

In [68]:
guild.runs().compare().sort_values(by="loss")[:10]

,run,operation,started,time,status,label,noise,x,step,loss
17,da896d5a,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.300000,0,-1.015978
18,da896d59,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.400000,0,-0.840888
5,e0ae06ac,train(),2019-05-23 16:15:24,0:00:00,completed,gp,0.1,-0.278346,0,-0.795046
16,da896d5b,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.200000,0,-0.773661
6,e0ae06ab,train(),2019-05-23 16:15:24,0:00:00,completed,gp,0.1,-0.186998,0,-0.754781
19,da896d58,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.500000,0,-0.620585
15,da896d5c,train(),2019-05-23 16:15:13,0:00:00,completed,,0.1,-0.100000,0,-0.526886
1,e1c6c6db,train(),2019-05-23 16:15:26,0:00:00,completed,gp,0.1,1.999571,0,-0.064552
7,e0ae06aa,train(),2019-05-23 16:15:24,0:00:00,completed,gp,0.1,2.000000,0,-0.036132
14,dc3e49a2,train(),2019-05-23 16:15:15,0:00:00,completed,random,0.1,1.249115,0,0.025076


## Next steps

This Notebook covers basic functionality provided by the `guild.ipy` module. The module lets you run functions and capture results as unique runs. You can view and compare run results using various module functions.

Generated runs can be further managed using the Guild command line interface. For more information, refer to [Guild AI Quick Start](https://guild.ai/docs/start/).